In [24]:
import requests
import json
import pandas as pd

In [25]:
placeholder = "#PLACEHOLDER_PARENT_METADATA_VALUE#"
acronym = "ISI"
scope2 = "f34e8dc2-5a79-45b3-a500-10f867ce3e9f"

# Instituts-Kürzel
#acronym = "IRB"
#scope2 = "fa1960d9-a52f-4a41-af94-2a2a40fa7e67" 

# Jahr von (min) bis (max). Für die Einschränkung auf ein Jahr, bitte den gleichen Wert für date_min und date_max einfügen
#date = "2021"
date_min = "2022" 
date_max = "2022"


url = "https://publica-rest.fraunhofer.de/server/api/discover/search/objects?query=institute:("+acronym+")&configuration=researchoutputs&f.dateIssued=%5B"+date_min+"%20TO%20"+date_max+"%5D,equals"

r = requests.get(url)
data = r.json()
anzahl = data["_embedded"]["searchResult"]["page"]["totalElements"]
anzahl = int(anzahl)
print(anzahl)
pages = data["_embedded"]["searchResult"]["page"]["totalPages"]
pages = int(pages)
#print(pages)
object = data["_embedded"]["searchResult"]["_embedded"]["objects"]

uuid = []
affil = []
authority = []
for j in range(len(object)):
    metadata = data["_embedded"]["searchResult"]["_embedded"]["objects"][j]["_embedded"]["indexableObject"]["metadata"]
    check = metadata.get("oairecerif.author.affiliation", 0)
    if check != 0:
        test = data["_embedded"]["searchResult"]["_embedded"]["objects"][j]["_embedded"]["indexableObject"]["metadata"]["oairecerif.author.affiliation"]
        for i in range(len(test)):
            uuid.append(data["_embedded"]["searchResult"]["_embedded"]["objects"][j]["_embedded"]["indexableObject"]["uuid"])
            affil.append(data["_embedded"]["searchResult"]["_embedded"]["objects"][j]["_embedded"]["indexableObject"]["metadata"]["oairecerif.author.affiliation"][i]["value"])
            authority.append(data["_embedded"]["searchResult"]["_embedded"]["objects"][j]["_embedded"]["indexableObject"]["metadata"]["oairecerif.author.affiliation"][i]["authority"])

if pages > 1:
    for k in range(pages-1):
        url = data["_embedded"]["searchResult"]["_links"]["next"]["href"]
        r = requests.get(url)
        data = r.json()
        object = data["_embedded"]["searchResult"]["_embedded"]["objects"]
        for j in range(len(object)):
            metadata = data["_embedded"]["searchResult"]["_embedded"]["objects"][j]["_embedded"]["indexableObject"]["metadata"]
            check = metadata.get("oairecerif.author.affiliation", 0)
            if check != 0:
                test = data["_embedded"]["searchResult"]["_embedded"]["objects"][j]["_embedded"]["indexableObject"]["metadata"]["oairecerif.author.affiliation"]
                for i in range(len(test)):
                    uuid.append(data["_embedded"]["searchResult"]["_embedded"]["objects"][j]["_embedded"]["indexableObject"]["uuid"])
                    affil.append(data["_embedded"]["searchResult"]["_embedded"]["objects"][j]["_embedded"]["indexableObject"]["metadata"]["oairecerif.author.affiliation"][i]["value"])
                    authority.append(data["_embedded"]["searchResult"]["_embedded"]["objects"][j]["_embedded"]["indexableObject"]["metadata"]["oairecerif.author.affiliation"][i]["authority"])
 
        
#Dataframe erstellen
df = pd.DataFrame({'uuid':uuid})
df['name'] = affil
df['authority'] = authority
#print(df)

#Löschen des eigenen Namens
df = df[df.authority != scope2]
#Löschen von Placeholdern
df = df[df.name != placeholder]

#Deduplizieren
df2 = df.drop_duplicates()

#Gewichten: goupby count
#df = df.groupby(['name', 'authority']).size()
df3 = df2.groupby(['name'])['name'].count().reset_index(name='counts')
df3 = df3.reset_index(drop=True)

print(df3)

297
                                                  name  counts
0                                               4Front       1
1                                       AINIA, Paterna       1
2                 AIT Austrian Institute of Technology       1
3                Aarhus Univ. Department of Management       1
4    Aarhus University,  Danish Centre for Studies ...       1
..                                                 ...     ...
302                   r2b energy consulting GmbH, Köln       1
303                                     viegand maagoe       2
304                                 Öko-Institut e. V.       8
305                          Öko-Institut e.V., Berlin       1
306                Öko-Institut e.V., Berlin, Freiburg       1

[307 rows x 2 columns]
